# Data Ingestion for Fake News Detection

This notebook demonstrates the data ingestion process for our fake news detection pipeline using Hive metastore tables. Data ingestion is the first critical step in any data science project, as it involves collecting, loading, and preparing the raw data for further processing.

## Why Data Ingestion is Important

In fake news detection, proper data ingestion ensures:
1. Data quality and consistency
2. Appropriate labeling of real and fake news articles
3. Balanced representation of both classes
4. Efficient storage for distributed processing

This notebook will guide you through the process of loading, combining, and processing news data using Apache Spark for distributed processing, with a focus on leveraging Hive metastore tables in Databricks for the complete dataset (approximately 45,000 articles).

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Import required libraries
import os
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, when, count, desc, rand
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Import our custom Hive data ingestion module
# Note: In Databricks, you may need to adjust this import path
# You can use %run ./hive_data_ingestion instead
import sys
sys.path.append('/dbfs/FileStore/tables')
from hive_data_ingestion import HiveDataIngestion

## Creating a Spark Session with Hive Support

We'll use Apache Spark for distributed data processing, with Hive support enabled to access the metastore tables. Let's create a properly configured Spark session optimized for the Databricks Community Edition limitations (1 driver, 15.3 GB Memory, 2 Cores).

In [ ]:
# Create a Spark session with configuration optimized for Databricks Community Edition
# - appName: Identifies this application in the Spark UI and logs
# - spark.sql.shuffle.partitions: Set to 8 (4x number of cores) for Community Edition
# - spark.driver.memory: Set to 8g to utilize available memory while leaving room for system
# - enableHiveSupport: Enables access to Hive metastore tables
spark = SparkSession.builder \
    .appName("FakeNewsDetection") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.driver.memory", "8g") \
    .enableHiveSupport() \
    .getOrCreate()

# Display Spark version information
print(f"Spark version: {spark.version}")
print(f"Spark configuration: {spark.sparkContext.getConf().getAll()}")

## Understanding Hive Metastore Tables

In Databricks, the Hive metastore provides a centralized repository to store metadata for tables and partitions. For our fake news detection project, we have two tables in the Hive metastore:

1. **`fake`**: Contains fake news articles with columns: title, text, subject, date
2. **`real`**: Contains real news articles with columns: title, text, subject, date

### Why Use Hive Tables?

Using Hive metastore tables offers several advantages over reading from CSV files:

1. **Centralized Metadata**: Schema information is stored centrally, ensuring consistency across sessions and users
2. **Optimized Performance**: Databricks optimizes query execution on Hive tables
3. **Access Control**: Tables can have access controls applied at the table level
4. **Persistence**: Tables persist across cluster restarts and sessions
5. **Catalog Integration**: Tables are visible in the Databricks catalog UI

Let's explore these tables using Spark SQL.

In [ ]:
# List all tables in the default database
print("Available tables in the Hive metastore:")
spark.sql("SHOW TABLES").show()

## Creating Directory Structure in DBFS

Let's create a directory structure in DBFS (Databricks File System) to organize our processed data files.

In [ ]:
def create_directory_structure():
    """Create directory structure for data storage in DBFS.
    
    This function creates the necessary directories for storing:
    - Combined data: The full dataset with both real and fake news
    - Processed data: Data after preprocessing steps
    - Model data: Data used for model training and evaluation
    - Sample data: Optional balanced samples for development with limited resources
    """
    # In Databricks, we use dbutils to interact with DBFS
    directories = [
        "dbfs:/FileStore/fake_news_detection/data/combined_data",
        "dbfs:/FileStore/fake_news_detection/data/processed_data",
        "dbfs:/FileStore/fake_news_detection/data/model_data",
        "dbfs:/FileStore/fake_news_detection/data/sample_data"
    ]
    
    for directory in directories:
        # Remove dbfs: prefix for dbutils.fs.mkdirs
        dir_path = directory.replace("dbfs:", "")
        dbutils.fs.mkdirs(dir_path)
        print(f"Created directory: {directory}")

# Create the directory structure
create_directory_structure()

## Loading and Exploring Data from Hive Tables

Now, let's load the data from Hive metastore tables and explore their structure. We'll use our custom `HiveDataIngestion` class to handle this process.

In [ ]:
# Initialize the HiveDataIngestion class
# We specify the table names in the Hive metastore
ingestion = HiveDataIngestion(spark, real_table="real", fake_table="fake")

# Load data from Hive tables
try:
    # This loads data from the Hive tables and registers them as temporary views
    real_df, fake_df = ingestion.load_data_from_hive()
    
except Exception as e:
    print(f"Error loading datasets from Hive: {str(e)}")

## Memory Management for Community Edition

Since we're working with the Databricks Community Edition (15.3 GB Memory, 2 Cores), we need to be careful about memory usage. Let's check the size of our datasets and implement memory-efficient processing strategies.

In [ ]:
# Check dataset sizes
real_count = real_df.count()
fake_count = fake_df.count()
total_count = real_count + fake_count

print(f"Real news dataset: {real_count} records")
print(f"Fake news dataset: {fake_count} records")
print(f"Total dataset size: {total_count} records")

# Memory management tips for Community Edition
print("\nMemory Management Tips for Databricks Community Edition:")
print("1. Process data in smaller batches when possible")
print("2. Use .unpersist() to release cached DataFrames when no longer needed")
print("3. Consider using sampling for exploratory analysis and model development")
print("4. Minimize the number of wide transformations (joins, groupBy, etc.)")

## Data Exploration with Spark SQL

Let's use Spark SQL to explore our datasets. SQL provides a familiar syntax for data exploration while leveraging Spark's distributed processing capabilities.

In [ ]:
# Explore real news data using Spark SQL
print("Sample of real news articles:")
spark.sql("""
    SELECT title, text, subject, date
    FROM true_news
    LIMIT 3
""").show(truncate=50)

# Explore fake news data using Spark SQL
print("\nSample of fake news articles:")
spark.sql("""
    SELECT title, text, subject, date
    FROM fake_news
    LIMIT 3
""").show(truncate=50)

# Count articles by subject in real news
print("\nReal news articles by subject:")
spark.sql("""
    SELECT subject, COUNT(*) as count
    FROM true_news
    GROUP BY subject
    ORDER BY count DESC
""").show()

# Count articles by subject in fake news
print("\nFake news articles by subject:")
spark.sql("""
    SELECT subject, COUNT(*) as count
    FROM fake_news
    GROUP BY subject
    ORDER BY count DESC
""").show()

## Analyzing Subject Distribution and Potential Data Leakage

Let's analyze the distribution of the 'subject' column across real and fake news to check if it might be a perfect discriminator between classes, which would indicate potential data leakage.

In [ ]:
# Analyze subject distribution across real and fake news
print("Analyzing subject distribution across classes...")

# First, let's combine the datasets with labels
real_df_with_label = real_df.withColumn("label", lit(1))  # 1 for real news
fake_df_with_label = fake_df.withColumn("label", lit(0))  # 0 for fake news
temp_combined = real_df_with_label.unionByName(fake_df_with_label)
temp_combined.createOrReplaceTempView("temp_combined")

# Check if any subjects appear in both real and fake news
print("\nSubjects that appear in both real and fake news:")
spark.sql("""
    SELECT subject, 
           SUM(CASE WHEN label = 1 THEN 1 ELSE 0 END) as real_count,
           SUM(CASE WHEN label = 0 THEN 1 ELSE 0 END) as fake_count
    FROM temp_combined
    GROUP BY subject
    HAVING real_count > 0 AND fake_count > 0
    ORDER BY real_count + fake_count DESC
""").show()

# Calculate correlation between subject and label
print("\nAnalyzing correlation between subject and label...")
# Convert subject to numeric using StringIndexer in a later step
# For now, let's check how many unique subjects are in each class
print("\nNumber of unique subjects in each class:")
spark.sql("""
    SELECT label, COUNT(DISTINCT subject) as unique_subjects
    FROM temp_combined
    GROUP BY label
    ORDER BY label DESC
""").show()

# Check if subject perfectly separates the classes
print("\nCombined dataset statistics:")
spark.sql("""
    SELECT label, COUNT(*) as count, COUNT(DISTINCT subject) as unique_subjects
    FROM temp_combined
    GROUP BY label
    ORDER BY label DESC
""").show()

# Show top subjects by class
print("\nSubject distribution by label:")
spark.sql("""
    SELECT 
        subject,
        SUM(CASE WHEN label = 1 THEN 1 ELSE 0 END) as real_count,
        SUM(CASE WHEN label = 0 THEN 1 ELSE 0 END) as fake_count
    FROM temp_combined
    GROUP BY subject
    ORDER BY real_count + fake_count DESC
    LIMIT 10
""").show()

# Release memory
temp_combined.unpersist()

## Data Leakage Warning

**Important**: Based on the analysis above, the 'subject' column appears to be a strong predictor of whether an article is real or fake news. This could indicate potential data leakage, as the subject categories might be directly related to the source of the news rather than the content itself.

### Recommendations:

1. **Consider removing the 'subject' column** from the feature set to ensure the model learns from the actual content rather than the source categorization.

2. **Alternatively, create two model variants** - one with and one without the 'subject' feature - to compare performance and understand the impact.

3. **Perform cross-validation** with careful stratification to ensure the model generalizes well across different subjects.

Let's proceed with adding labels and combining the datasets, keeping this potential data leakage in mind.

## Adding Labels and Combining Datasets

Now, let's add labels to our datasets (1 for real news, 0 for fake news) and combine them into a single dataset. This labeling is crucial for training machine learning models to distinguish between real and fake news.

In [ ]:
# Combine datasets with labels using our ingestion class
combined_df = ingestion.combine_datasets(real_df, fake_df)

# Show distribution of subjects across labels
print("\nSubject distribution by label:")
spark.sql("""
    SELECT 
        subject,
        SUM(CASE WHEN label = 1 THEN 1 ELSE 0 END) as real_count,
        SUM(CASE WHEN label = 0 THEN 1 ELSE 0 END) as fake_count
    FROM combined_news
    GROUP BY subject
    ORDER BY real_count + fake_count DESC
    LIMIT 10
""").show()

## Saving Combined Dataset

Let's save the combined dataset in Parquet format in DBFS, which is optimized for distributed processing. We'll also save it as a Hive table for easier access. We'll use partitioning to optimize query performance on the full dataset.

In [ ]:
# Save combined dataset in Parquet format to DBFS
print("Saving combined dataset to DBFS...")
ingestion.save_to_parquet(
    combined_df, 
    "dbfs:/FileStore/fake_news_detection/data/combined_data/combined_news.parquet",
    partition_by="label"
)

# Save combined dataset as a Hive table
print("Saving combined dataset as a Hive table...")
ingestion.save_to_hive_table(
    combined_df,
    "combined_news",
    partition_by="label"
)

# Verify the saved data
print("\nVerifying saved data...")
saved_df = spark.read.parquet("dbfs:/FileStore/fake_news_detection/data/combined_data/combined_news.parquet")
print(f"Saved dataset record count: {saved_df.count()}")

# Check partition structure
print("\nPartition structure:")
spark.sql("""
    SELECT label, COUNT(*) as count
    FROM parquet.`dbfs:/FileStore/fake_news_detection/data/combined_data/combined_news.parquet`
    GROUP BY label
    ORDER BY label DESC
""").show()

# Release memory
saved_df.unpersist()

## Creating a Development Sample (Optional)

For development and testing with limited resources in the Community Edition, we can create a balanced sample dataset. This is particularly useful for model development and testing, where the full dataset might be too large for efficient iteration.

In [ ]:
# Create a balanced sample for development and testing
print("Creating balanced sample for development...")
sample_size = 1000  # Adjust based on your memory constraints

# Use stratified sampling to maintain class distribution
sample_fractions = {
    0: min(1.0, sample_size/fake_count),
    1: min(1.0, sample_size/real_count)
}

sample_df = combined_df.sampleBy("label", fractions=sample_fractions, seed=42)

# Save sample to DBFS
print("Saving sample to DBFS...")
sample_df.write \
    .mode("overwrite") \
    .partitionBy("label") \
    .option("compression", "snappy") \
    .parquet("dbfs:/FileStore/fake_news_detection/data/sample_data/sample_news.parquet")

# Save sample as Hive table
print("Saving sample as Hive table...")
sample_df.write.mode("overwrite").saveAsTable("sample_news")

# Verify sample
print("\nSample dataset statistics:")
spark.sql("""
    SELECT label, COUNT(*) as count
    FROM sample_news
    GROUP BY label
    ORDER BY label DESC
""").show()

print("\nNote: This sample is intended for development and testing only.")
print("For production models, use the full dataset or larger samples as resources permit.")

## Community Edition Resource Management

When working with the Databricks Community Edition (15.3 GB Memory, 2 Cores), consider these resource management strategies:

In [ ]:
# Display resource management tips
print("Resource Management Tips for Databricks Community Edition:")
print("\n1. Spark Configuration:")
print("   - Use spark.sql.shuffle.partitions = 8 (4x number of cores)")
print("   - Allocate more memory to driver (8g) than executors")
print("   - Avoid excessive caching of large DataFrames")

print("\n2. Data Processing:")
print("   - Use .unpersist() to release memory when DataFrames are no longer needed")
print("   - Process data in smaller batches when possible")
print("   - Use sampling for exploratory analysis and initial model development")

print("\n3. Model Training:")
print("   - Use stratified sampling for complex models (LSTM, deep learning)")
print("   - Consider simpler models for the full dataset (Naive Bayes, Logistic Regression)")
print("   - Implement cross-validation with smaller fold counts (3 instead of 5)")

print("\n4. Storage Optimization:")
print("   - Use Parquet format with Snappy compression")
print("   - Partition data by frequently filtered columns (e.g., label)")
print("   - Use Hive tables for efficient metadata management")

## Cleanup

Let's clean up our session to free resources.

In [ ]:
# Release memory
if 'combined_df' in locals():
    combined_df.unpersist()
if 'real_df' in locals():
    real_df.unpersist()
if 'fake_df' in locals():
    fake_df.unpersist()
if 'sample_df' in locals():
    sample_df.unpersist()

print("Data ingestion completed successfully!")
print("The data is now ready for preprocessing and feature engineering.")